In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from integral_timber_joints.planning import load_pddlstream

Using pddlstream from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pddlstream\pddlstream\__init__.py


## Parse ITJ process

In [3]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, \
    get_process_path, save_process

In [4]:
# parse options
design_dir = '210916_SymbolicPlanning' # '210419_AnticlasticShelter' # '210605_ScrewdriverTestProcess' # 210419_AnticlasticShelter
problem = 'nine_pieces_process.json' #'shelter_process.json' # 'shelter_process.json' pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = '.'

In [5]:
process = parse_process(design_dir, problem, subdir=problem_subdir)

# Double check entire solution is valid
for beam_id in process.assembly.sequence:
    if not process.dependency.beam_all_valid(beam_id):
        process.dependency.compute_all(beam_id)
        assert process.dependency.beam_all_valid(beam_id)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210916_SymbolicPlanning\nine_pieces_process.json


## Construct PDDL problem

In [6]:
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

In [7]:
debug = 0
verbose = 1
use_partial_order = 1

In [8]:
pddlstream_problem = get_pddlstream_problem(process, use_partial_order=use_partial_order, debug=debug)

In [9]:
print('Init:', pddlstream_problem.init)
print('Goal:', pddlstream_problem.goal)

Init: [('RobotConf', Conf-home), ('RobotAtConf', Conf-home), 'CanMove', 'RobotToolChangerEmpty', ('Element', 'b0'), ('AtRack', 'b0'), ('IsElement', 'b0'), ('Grounded', 'b0'), ('Element', 'b1'), ('AtRack', 'b1'), ('IsElement', 'b1'), ('Grounded', 'b1'), ('Element', 'b2'), ('AtRack', 'b2'), ('IsElement', 'b2'), ('Element', 'b3'), ('AtRack', 'b3'), ('IsElement', 'b3'), ('Element', 'b5'), ('AtRack', 'b5'), ('IsElement', 'b5'), ('Grounded', 'b5'), ('Element', 'b6'), ('AtRack', 'b6'), ('IsElement', 'b6'), ('Element', 'b7'), ('AtRack', 'b7'), ('IsElement', 'b7'), ('Grounded', 'b7'), ('Element', 'b8'), ('AtRack', 'b8'), ('IsElement', 'b8'), ('Element', 'b4'), ('AtRack', 'b4'), ('IsElement', 'b4'), ('Joint', 'b0', 'b2'), ('NoToolAtJoint', 'b0', 'b2'), ('Joint', 'b0', 'b3'), ('NoToolAtJoint', 'b0', 'b3'), ('Joint', 'b0', 'b6'), ('NoToolAtJoint', 'b0', 'b6'), ('Joint', 'b1', 'b2'), ('NoToolAtJoint', 'b1', 'b2'), ('Joint', 'b1', 'b3'), ('NoToolAtJoint', 'b1', 'b3'), ('Joint', 'b1', 'b8'), ('NoTool

## Solve

In [10]:
from pddlstream.algorithms.downward import set_cost_scale, parse_action
from pddlstream.algorithms.meta import solve
from pddlstream.utils import INF

pddlstream_problem = get_pddlstream_problem(process, use_partial_order=use_partial_order, debug=debug)

costs = True
set_cost_scale(1)
success_cost = 0 if costs else INF

# discrete_planner = {
#     'search': 'eager', # eager | lazy
#     # lazy might be faster but the solution quality might be low
#     'evaluator': 'greedy',
#     'heuristic': 'goal', # goal | add | ff
#     #'heuristic': ['ff', get_bias_fn(element_from_index)],
#     'successors': 'all', # all | random
#     # tiebreaker
#     #'successors': get_order_fn(element_from_index),
#     # ? ordering scaffolds can come here
#     # TODO: confirm that this is working correctly
# }
solution = solve(pddlstream_problem, algorithm='incremental', 
                 max_time=600,
                 success_cost=success_cost, unit_costs=not costs,
                 max_planner_time=300, debug=debug, verbose=True)

Warning! All actions have no cost. Recommend setting unit_costs=True
Eager Calls: 0 | Successes: 0 | Results: 0 | Counts: {}
Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 111 | Solved: False | Cost: inf | Search Time: 0.010 | Sample Time: 0.000 | Time: 0.010
iter=inf, outs=1) sample-move:(Conf-home, Conf-home)->[(Traj)]
iter=inf, outs=1) sample-pick-element:(b0)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-place-element:(b0)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-pick-element:(b1)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-place-element:(b1)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-pick-element:(b2)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-place-element:(b2)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-pick-element:(b3)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-place-element:(b3)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-pick-element:(b5)->[(Conf, Conf, Traj)]
iter=inf, outs=1) sample-place-element:(b5)->[(Conf, Conf, Traj)]
iter=inf, outs

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf-home, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf-home)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf-home, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf

...

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf-home)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf-home)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf

iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf-home)->[(Traj)]
iter=inf, outs=1) sample-move:(Conf, Conf

In [26]:
from pddlstream.language.constants import print_plan, is_plan

plan, _, facts = solution
# print('-'*10)
print_plan(plan)
assert is_plan(plan), 'planning fails.'

# actions = [action for action in plan if action.name == 'add_element' or action.name == 'add_scaffold_element']
# element_sequence = [ac.args[2] for ac in actions]

AssertionError: planning fails.

In [162]:
element_sequence

[0, 5, 6, 1, 7, 2, 8, 3, 4]